In [11]:
import numpy as np
import pandas as pd
import pickle
import lightgbm as lgb
import sklearn as sk
from sklearn import ensemble

In [12]:
x_val_list = pickle.load(open('../gen_data/test_data_enriched--enrich1.ipynb--.pickle','rb'))
x_train_list = pickle.load(open('../gen_data/x_train--simple_validation_split.ipynb--.pickle','rb'))
y_train_list = pickle.load(open('../gen_data/y_train--simple_validation_split.ipynb--.pickle','rb'))
y_val_list = pickle.load(open('../gen_data/y_val--simple_validation_split.ipynb--.pickle','rb'))
test0 = pd.read_csv('../original_data/test.csv.gz')

In [13]:
RUN_ON_TEST=False
FILE_NUM = 2

In [14]:
#assign variables to full train - test split
if RUN_ON_TEST:
    x_train = x_train_list.pop()
    y_train = y_train_list.pop()
    x_val = x_val_list.pop()
    y_val = y_val_list.pop()
else:
    # with train - validation split
    x_train = x_train_list[0]
    y_train = y_train_list[0]
    x_val = x_val_list[0]
    y_val = y_val_list[0]

In [15]:
#write these files to analyse in R to make sure theyre correct

if RUN_ON_TEST:
    x_val.to_csv('../gen_data/x_val_testfold.csv',index=False,na_rep=np.nan)
    y_val.to_csv('../gen_data/y_val_testfold.csv',index=False,na_rep=np.nan)
    x_train.to_csv('../gen_data/x_train_testfold.csv',index=False,na_rep=np.nan)
    y_train.to_csv('../gen_data/y_train_testfold.csv',index=False,na_rep=np.nan)
else:
    x_val.to_csv('../gen_data/x_val_valfold.csv',index=False,na_rep=np.nan)
    y_val.to_csv('../gen_data/y_val_valfold.csv',index=False,na_rep=np.nan)
    x_train.to_csv('../gen_data/x_train_valfold.csv',index=False,na_rep=np.nan)
    y_train.to_csv('../gen_data/y_train_valfold.csv',index=False,na_rep=np.nan)


In [ ]:
#model specific preprocessing
def model1_preproc(x,*args):
    x = x.fillna(999)
    if len(args):
        y = args[0]
        return([x.drop(['shop_name','item_name','item_category_name'],axis=1),y])
    else:
        return(x.drop(['shop_name','item_name','item_category_name'],axis=1))

In [ ]:
xtp , ytp = model1_preproc(x_train,y_train) 
model = ensemble.RandomForestRegressor(verbose=10,n_jobs=4,n_estimators=10,random_state=0)
model.fit(xtp,ytp)

In [ ]:
xvp  = model1_preproc(x_val)

In [ ]:
preds = model.predict(xvp)

In [ ]:
np.sqrt(np.mean((preds-y_val)**2))
#4.683318375274549

In [ ]:
x_val.head()

In [ ]:
if RUN_ON_TEST:
    submission = x_val.copy()
    submission['item_cnt_month'] = preds
    submission.drop(['date_block_num','item_category_id','shop_name','item_name','item_category_name'],axis=1,inplace=True)
    submission = test0.merge(submission,'right',on=['shop_id','item_id']).drop(['shop_id','item_id'],axis=1)
    submission.to_csv('../gen_data/model1sub_'+str(FILE_NUM)+'_--model1_script1.ipynb--.csv',index=False)
    